<a href="https://colab.research.google.com/github/jjangmo91/Cervus-nippon/blob/main/Cervus_nippon_prep_st.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. 환경 설정 및 라이브러리 임포트

In [ ]:
# 필수 패키지 설치
!pip install eesdm -q
!pip install geojson -q
!pip install geemap -U -q

# 라이브러리 임포트
import ee
import geemap
import pandas as pd
import eeSDM
from ipyleaflet import WidgetControl
from ipywidgets import Label
import glob

# Earth Engine 인증 및 초기화
ee.Authenticate()
ee.Initialize(project='ee-jjangmo91')

# Google Drive 마운트
from google.colab import drive
drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/KNPS/Deer/Ecotopia_Data_2024_2025/'

# 2. Spatial Thinnging

In [ ]:
# 공간 솎아내기(Spatial Thinning) 함수 정의
def spatial_thinning(df, resolution_m, lon_col='Longitude', lat_col='Latitude'):
    """
    주어진 해상도(미터 단위)에 따라 격자 기반 공간 솎아내기를 수행합니다.
    각 격자 셀 내에서 첫 번째 좌표만 남깁니다.
    """
    if df.empty:
        return df

    # 1도는 약 111km라는 근사치를 사용하여 미터 해상도를 위경도 단위로 변환
    res_deg = resolution_m / 111000

    # 각 좌표가 속한 격자의 고유 ID 생성
    df['grid_id'] = (df[lat_col] // res_deg).astype(str) + '_' + (df[lon_col] // res_deg).astype(str)

    # 각 격자 ID별로 첫 번째 데이터만 남기고 중복 제거
    thinned_df = df.drop_duplicates(subset='grid_id', keep='first').copy()

    # 임시로 사용한 grid_id 열 삭제 후 반환
    return thinned_df.drop(columns=['grid_id'])

# 솎아내기 해상도 설정 (단위: 미터)
thinning_resolution_meters = 300

In [ ]:
# 개체별 GPS 파일 목록 불러오기
dsf_files = sorted(glob.glob(data_path + 'DSF-*.csv'))
dsm_files = sorted(glob.glob(data_path + 'DSM-*.csv'))
print(f"암컷(DSF) 파일 {len(dsf_files)}개, 수컷(DSM) 파일 {len(dsm_files)}개를 발견했습니다.")

# 데이터 통합 함수 정의
def combine_gps_data(file_list):
    df_list = [pd.read_csv(file) for file in file_list]
    return pd.concat(df_list, ignore_index=True)

# 성별로 데이터 통합
df_female = combine_gps_data(dsf_files)
df_male = combine_gps_data(dsm_files)
df_all = pd.concat([df_female, df_male], ignore_index=True)

# 시간 정보(Collecting_time)를 날짜/시간 형태로 변환
for df in [df_female, df_male, df_all]:
    df['Collecting_time'] = pd.to_datetime(df['Collecting_time'], errors='coerce')

# 분석 그룹(전체, 암컷, 수컷)별로 반복 처리
datasets_to_create = {'all': df_all, 'female': df_female, 'male': df_male}

for name, df in datasets_to_create.items():
    # 전체 기간 데이터에 공간 솎아내기 적용
    print(f"\n[{name.upper()} 전체 기간] 원본 좌표 수: {len(df):,}")
    df_entire_thinned = spatial_thinning(df.copy(), thinning_resolution_meters)
    print(f"[{name.upper()} 전체 기간] 솎아내기 후 좌표 수: {len(df_entire_thinned):,} ({thinning_resolution_meters}m 해상도)")

    # 솎아낸 전체 기간 데이터셋 저장
    sdm_entire = pd.DataFrame({
        'longitude': df_entire_thinned['Longitude'],
        'latitude': df_entire_thinned['Latitude'],
        'cervus': 'cervus-nippon'
    })
    # 파일 이름에 해상도 정보를 추가하여 구분하기 용이하게 만듭니다.
    output_filename_entire = f'sdm_occurrences_{name}_entire_thinned_{thinning_resolution_meters}m.csv'
    sdm_entire.to_csv(data_path + output_filename_entire, index=False)
    print(f"   => [{name.upper()} 전체 기간] 솎아낸 좌표 저장 완료.")

    # 겨울철 데이터에 공간 솎아내기 적용
    df_winter = df[df['Collecting_time'].dt.month.isin([12, 1, 2])].copy()
    if not df_winter.empty:
        print(f"[{name.upper()} 겨울철] 원본 좌표 수: {len(df_winter):,}")
        df_winter_thinned = spatial_thinning(df_winter, thinning_resolution_meters)
        print(f"[{name.upper()} 겨울철] 솎아내기 후 좌표 수: {len(df_winter_thinned):,} ({thinning_resolution_meters}m 해상도)")

        # 솎아낸 겨울철 데이터셋 저장
        sdm_winter = pd.DataFrame({
            'longitude': df_winter_thinned['Longitude'],
            'latitude': df_winter_thinned['Latitude'],
            'cervus': 'cervus-nippon'
        })
        output_filename_winter = f'sdm_occurrences_{name}_winter_thinned_{thinning_resolution_meters}m.csv'
        sdm_winter.to_csv(data_path + output_filename_winter, index=False)
        print(f"   => [{name.upper()} 겨울철] 솎아낸 좌표 저장 완료.")

#3. 전처리된 4개 그룹 데이터 시각화

In [ ]:
try:
    df_female_entire = pd.read_csv(data_path + f'sdm_occurrences_female_entire_thinned_{thinning_resolution_meters}m.csv')
    df_male_entire = pd.read_csv(data_path + f'sdm_occurrences_male_entire_thinned_{thinning_resolution_meters}m.csv')
    df_female_winter = pd.read_csv(data_path + f'sdm_occurrences_female_winter_thinned_{thinning_resolution_meters}m.csv')
    df_male_winter = pd.read_csv(data_path + f'sdm_occurrences_male_winter_thinned_{thinning_resolution_meters}m.csv')
    print("Visualization data for 4 thinned groups loaded successfully.")
except FileNotFoundError:
    print("Error: Thinned data files not found. Please ensure the thinning process ran correctly.")
    # 파일이 없는 경우, 이후 코드 실행을 막기 위해 빈 데이터프레임 생성
    df_female_entire, df_male_entire, df_female_winter, df_male_winter = [pd.DataFrame(columns=['longitude', 'latitude'])]*4

# 속리산 국립공원 경계(AOI) 불러오기 (WDPA ID: 773)
protected_areas = ee.FeatureCollection("WCMC/WDPA/current/polygons")
aoi = protected_areas.filter(ee.Filter.eq('WDPAID', 773)).geometry()
print("Songnisan National Park AOI loaded.")
print("-" * 30)

# geemap 호환성을 위해 모든 데이터프레임의 열 이름을 변경
for df in [df_female_entire, df_male_entire, df_female_winter, df_male_winter]:
    if not df.empty:
      df.rename(columns={'x': 'longitude', 'y': 'latitude'}, inplace=True)

# pandas 데이터프레임을 ee.FeatureCollection으로 변환
ee_f_entire = geemap.pandas_to_ee(df_female_entire)
ee_m_entire = geemap.pandas_to_ee(df_male_entire)
ee_f_winter = geemap.pandas_to_ee(df_female_winter)
ee_m_winter = geemap.pandas_to_ee(df_male_winter)

# geemap을 이용한 대화형 지도 생성
Map = geemap.Map(center=[36.54, 127.85], zoom=11)

# 지도에 레이어 추가
Map.addLayer(aoi, {'color': '#006600', 'fillColor': '#33996655'}, 'Songnisan_NP_AOI')
Map.addLayer(ee_f_entire, {'color': '#F08080'}, 'Female (Entire) - Thinned')
Map.addLayer(ee_m_entire, {'color': '#87CEEB'}, 'Male (Entire) - Thinned')
Map.addLayer(ee_f_winter, {'color': '#DC143C'}, 'Female (Winter) - Thinned')
Map.addLayer(ee_m_winter, {'color': '#0000CD'}, 'Male (Winter) - Thinned')

# 범례(Legend) 추가
Map.add_legend(
    title="Legend (Thinned Data)",
    legend_dict={
        f"Female (Entire)": "F08080",
        f"Male (Entire)": "87CEEB",
        f"Female (Winter)": "DC143C",
        f"Male (Winter)": "0000CD"
    }
)

# 제목 추가
title_widget = Label(value=f"Sika Deer Occurrence ({thinning_resolution_meters}m Thinned)")
Map.add_control(WidgetControl(widget=title_widget, position='topright'))

# 지도 출력
display(Map)